В семинаре мы работали с датасетами инструкций alpaca и dolly. Они англоязычные. В домашке вам нужно создать аналогичный датасет на русском языке и обучить аналогичную модель на этом датасете. В качестве итогового результата у вас должна получится модель, которая может связно отвечать на русскоязычные инструкции на русском языке. Приведите как минимум три разных примера. Правильность ответов не так важна, так как вы скорее всего будете использовать небольшие модели, но текст должен быть не рандомным.

Русскоязычный датасет инструкций должен быть больше 5 тысяч примеров. Он может быть основнован на alpaca/dolly (например, вы можете просто прогнать все через переводную модель, которая была на семинаре, или даже google translate). Или вы можете придумать способ создать аналогичный датасет каким-то другим способом (переделать открытые датасеты с помощью правил). Датасет может быть не уникальным, можно скооперироваться с одногруппниками и сделать один датасет на всех.

Вы можете попробовать дообучать любую небольшую decoder-only модель. Скорее всего лучше всего будут работать модели, изначально обученные на русском языке (rugpt например). Но возможно даже модели вроде opt можно будет дообучить на русскоязычных инструкциях.


In [1]:
!pip install requests==2.31

In [2]:
!pip install zstandard jsonlines datasets
import datasets
from datasets import load_dataset


dataset = load_dataset("IlyaGusev/ru_turbo_alpaca")

/home/aslyutikov/Masha/venv/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
!pip install torch==2.0.1

In [4]:
!pip install transformers

In [5]:
from typing import Optional, Dict, Sequence
import transformers
import copy
import logging
from dataclasses import dataclass, field
import torch
from torch.utils.data import Dataset
from transformers import Trainer

In [6]:
!pip install accelerate -U

In [7]:
import accelerate
accelerate.__version__

'0.29.3'

In [8]:
!pip install transformers[torch]

In [9]:
dataset['train'][1]

{'instruction': 'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
 'input': '',
 'output': 'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и украшениями.',
 'alternative_output': 'Для изготовления торта необходимо следующее:\n\nИнгредиенты для теста:\n\n- Мука\n- Сахар\n- Яйца\n- Растительное масло или сливочное масло\n- Разрыхлитель, как правило, это порошок для выпечки\n- Молоко\n\nИнгредиенты для крема:\n\n- Сливочное масло\n- Сахарная пудра\n- Ванилин или другой ароматизатор\n- Молоко или сливки\n- Фрукты или ягоды (если используются)\n\nШаги изготовления торта:\n\n1. Для теста смешайте муку, сахар и разрыхли

Код с пары (но русскоязычный промпт):

In [10]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"
PROMPT_DICT = {
    "prompt_input": (
        "Ниже представлена инструкция с описанием задания, а также входные данные с дополнительной информацией. "
        "Напишите подходящий ответ на запрос.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Ниже представлена инструкция с описанием задания. "
        "Напишите подходящий ответ на запрос.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [11]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [12]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [13]:
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = data["train"]

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [14]:
model_name = "facebook/opt-125m"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

/home/aslyutikov/Masha/venv/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

In [16]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data=dataset)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [17]:
train_args = transformers.TrainingArguments(learning_rate=1e-5,
                 num_train_epochs=2,
                 per_device_train_batch_size=2,
                 gradient_accumulation_steps=1,
                 evaluation_strategy='no',
                 weight_decay=0.,
                 warmup_ratio=0.03,
                 lr_scheduler_type="cosine",
                 save_strategy='no',
                 logging_steps=1000,
                 output_dir="opt125_instruct_ft")

In [18]:
trainer = Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

In [19]:
trainer.train()

Step,Training Loss
1000,1.296100
2000,1.196200
3000,1.132100
4000,1.112100
5000,1.087700
6000,1.058600
7000,1.044300
8000,1.027000
9000,1.009700
10000,0.998300


TrainOutput(global_step=29822, training_loss=0.9736383548245633, metrics={'train_runtime': 3205.6547, 'train_samples_per_second': 18.606, 'train_steps_per_second': 9.303, 'total_flos': 1.4688052883712e+16, 'train_loss': 0.9736383548245633, 'epoch': 2.0})

In [20]:
trainer.save_model('opt125_ft_02')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warning will be raised to an exception in v4.41.


In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM
MODEL_NAME = 'opt125_ft_02'

In [22]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

Демонстрация работы модели:

In [23]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Below is an instruction that describes a task, paired with an input that provides further context. "
              "Write a response that appropriately completes the request.\n\n"
              f"### Instruction:\n{instruction}\n\n### Input:\n{text}\n\n### Response:")

    inputs = tokenizer([prompt],
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        temperature=0.4,
        max_length=1000,
        no_repeat_ngram_size=15,
        repetition_penalty= 0.5,
        early_stopping=False,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=True,
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=False)
    return summaries[0]

In [24]:
instruction = "Решите уравнение, найдите x"
text = "2x - 4 = 0"
predict_for_instruction(instruction, text, model)

/home/aslyutikov/Masha/venv/lib/python3.10/site-packages/transformers/generation/utils.py:1256: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'x = 4</s>'

In [25]:
instruction = "Напишите короткое сочинение о любви"
text = ""
predict_for_instruction(instruction, text, model)

'любви прошение о любви прошее обработке прошенип прошенип приростности.</s>'

In [26]:
instruction = "Какое сегодня число?"
text = ""
predict_for_instruction(instruction, text, model)

'Какие сегодня чичели?</s>'

In [27]:
instruction = "Продолжите стихотворение"
text = "Муха села на варенье"
predict_for_instruction(instruction, text, model)

'Муха села находится варенье.</s>'

In [28]:
instruction = "Напиши письмо начальнику, сообщающее о том, что ты опаздываешь на работу"
text = ""
predict_for_instruction(instruction, text, model)

'Начника на работу мобильно помогает мне расслабитьсь на прошлом гомаде на работе. Начника начника помогает мне помочь начать с прошлого гомада на работе, но она помогает мне улучшить нашу гомаду на работе. Но она помогает улучшить н Travis, Travis начника, следует привести от работы на работе.</s>'